In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import os
import shutil
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from ultralytics import YOLO
import ultralytics
import yaml

In [2]:
train_data= os.path.join(os.path.abspath("./archive"), "images", "images")

In [3]:
df= pd.read_csv('./archive/labels_train.csv')
df

,frame,xmin,xmax,ymin,ymax,class_id
0,1478019952686311006.jpg,237,251,143,155,1
1,1478019952686311006.jpg,437,454,120,186,3
2,1478019953180167674.jpg,218,231,146,158,1
3,1478019953689774621.jpg,171,182,141,154,2
4,1478019953689774621.jpg,179,191,144,155,1
...,...,...,...,...,...,...
132401,1479506176491553178.jpg,166,186,139,156,1
132402,1479506176491553178.jpg,182,204,142,153,1
132403,1479506176491553178.jpg,239,261,139,155,1
132404,1479506176491553178.jpg,259,280,139,157,1


In [4]:
df['class_id'].unique()

array([1, 3, 2, 5, 4])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132406 entries, 0 to 132405
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   frame     132406 non-null  object
 1   xmin      132406 non-null  int64 
 2   xmax      132406 non-null  int64 
 3   ymin      132406 non-null  int64 
 4   ymax      132406 non-null  int64 
 5   class_id  132406 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 6.1+ MB


In [6]:
labels= {0:'car',1:'truck',2:'person',3:'bicycle',4:'traffic_light'}

In [7]:
df.head()

,frame,xmin,xmax,ymin,ymax,class_id
0,1478019952686311006.jpg,237,251,143,155,1
1,1478019952686311006.jpg,437,454,120,186,3
2,1478019953180167674.jpg,218,231,146,158,1
3,1478019953689774621.jpg,171,182,141,154,2
4,1478019953689774621.jpg,179,191,144,155,1


In [8]:
width = 480
height = 300

df['x_center'] = (df['xmin'] + df['xmax']) / 2
df['y_center'] = (df['ymin'] + df['ymax']) / 2
df['width'] = (df['xmax'] - df['xmin'])
df['height'] = (df['ymax'] - df['ymin'])
df['class_id'] = df['class_id'] - 1

# normalizing the bounding box coordinates for yolo format
df["x_center"]=(df["x_center"]/width).round(3)
df["y_center"]=(df["y_center"]/height).round(3)
df["width"]=(df["width"]/width).round(3)
df["height"]=(df["height"]/height).round(3)

df_yolo=df[["frame","class_id","x_center","y_center","width","height"]]
df_yolo.head()

,frame,class_id,x_center,y_center,width,height
0,1478019952686311006.jpg,0,0.508,0.497,0.029,0.040
1,1478019952686311006.jpg,2,0.928,0.510,0.035,0.220
2,1478019953180167674.jpg,0,0.468,0.507,0.027,0.040
3,1478019953689774621.jpg,1,0.368,0.492,0.023,0.043
4,1478019953689774621.jpg,0,0.385,0.498,0.025,0.037


In [9]:
root_dir="./data"
labels_dir="./data/labels"
images_dir="./data/images"

In [10]:
img_list=list(sorted(os.listdir(train_data)))
print(f"Total images: {train_data} -> {len(img_list)}")


Total images: d:\_egyetem\halado_digikep\archive\images\images -> 22241


In [ ]:
df_yolo_set = set(df_yolo['frame'])

# Precompute the subset assignments
val_percent = 15 / 100.0
test_percent = 15 / 100.0
train_percent = (100 - val_percent - test_percent) / 100.0
subset_assignments = []
for i in range(len(img_list)):
    if i < train_percent / 100 * len(img_list):
        subset_assignments.append("train")
    elif i < (train_percent + val_percent) / 100 * len(img_list):
        subset_assignments.append("val")
    else:
        subset_assignments.append("test")

def process_image(i, img_name):
    subset = subset_assignments[i]
    
    if img_name in df_yolo_set:
        columns = ["class_id", "x_center", "y_center", "width", "height"]
        img_boxes = df[df['frame'] == img_name][columns].values  # All rows matching the img_name
        label_path = os.path.join(labels_dir, subset, img_name[:-4] + ".txt")
        
        os.makedirs(os.path.dirname(label_path), exist_ok=True)  # Ensure directory exists

        with open(label_path, "w") as f:
            for row in img_boxes:
                text = " ".join(row.astype(str))
                f.write(text + "\n")
    
    old_image_path = os.path.join(train_data, img_name)
    new_image_path = os.path.join(images_dir, subset, img_name)
    os.makedirs(os.path.dirname(new_image_path), exist_ok=True)  # Ensure directory exists
    shutil.copy(old_image_path, new_image_path)


with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(lambda p: process_image(*p), enumerate(img_list))


In [ ]:
def display_file_content(file_path):
  """Displays the content of a text file."""
  try:
    with open(file_path, 'r') as file:
      content = file.read()
      #print(content)
      
    return content
  
  except FileNotFoundError:
    #print(f"File not found: {file_path}")  
    return False

found = 0
not_found = 0
first = True
# Example usage:
for fname in img_list:
  for subset in ['train', 'val', 'test']:
    res = display_file_content(os.path.join(labels_dir, subset, fname[:-4] + ".txt"))
    if res:
      found += 1
      if first:
        print(res)
        first = False
      break
    
    if subset == 'test':
      not_found += 1

print(f"Found labels for {found} images, not found for {not_found} images. There were {len(img_list)} images in total.")



0.0 0.508 0.497 0.029 0.04
2.0 0.928 0.51 0.035 0.22

Found labels for 18000 images, not found for 4241 images. There were 22241 images in total.


In [11]:
ultralytics.checks()

Ultralytics 8.3.221  Python-3.13.1 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Setup complete  (16 CPUs, 15.2 GB RAM, 704.2/953.9 GB disk)


In [12]:
yolo_format=dict(path=".",
                 train="./data/images/train",
                 val="./data/images/val",
                 nc=5,
                 names={0:"car",1:'truck',2:'person',3:'bicycle',4:'traffic_light'})
             
with open('./data/data.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [16]:
import torch
model= YOLO('yolo11n.pt')
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
model.to(DEVICE)

Using device: cuda


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [17]:
results = model.train(
                data='./data/data.yaml', 
                epochs=7, 
                imgsz=640,
                batch=8,
                workers = 2
        )

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=7, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False,

SystemError: Caught SystemError in DataLoader worker process 0.
Original numpy._core._exceptions._ArrayMemoryError: Unable to allocate 1.17 MiB for an array with shape (640, 640, 3) and data type uint8

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\_egyetem\halado_digikep\.venv\Lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "d:\_egyetem\halado_digikep\.venv\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "d:\_egyetem\halado_digikep\.venv\Lib\site-packages\ultralytics\data\base.py", line 381, in __getitem__
    return self.transforms(self.get_image_and_label(index))
           ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\_egyetem\halado_digikep\.venv\Lib\site-packages\ultral